<font size = 2> <B> PHILIP AUERBACH <BR>UNI: PJA2113 </B></font>

<font size = 2> <B> QOG Analysis </B></font><br> <B> Lab #2 </B> <br> Description of Project: <br> The goal of the project was to utilize multiple variables from the QOG databse and derive some understanding of the complex data, and try to find variables that work well together to develop / confirm concepts about the world. 

<b> Project Setup: </b>

In [55]:
# Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf 

#Grabbing Data
url = 'https://www.qogdata.pol.gu.se/data/qog_std_cs_jan25.xlsx'
df = pd.read_excel(url)

#Visualize 5 first rows of data
df.head()

,cname_qog,cname,ccode_qog,ccodecow,ccodealp,ccode,aii_acc,aii_aio,aii_cilser,aii_elec,...,yri_fem40,yri_fem4160,yri_fem61,yri_meanage,yri_medianage,yri_mp30,yri_mp35,yri_mp40,yri_mp4160,yri_mp61
0,Afghanistan,Afghanistan,4,700.0,AFG,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,Albania,8,339.0,ALB,8.0,NaN,NaN,NaN,NaN,...,20.8,75.0,4.2,51.0,51.0,0.0,5.8,15.9,72.5,11.6
2,Algeria,Algeria,12,615.0,DZA,12.0,12.50,5.0,12.5,8.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andorra,Andorra,20,232.0,AND,20.0,NaN,NaN,NaN,NaN,...,7.1,78.6,14.3,48.2,48.5,3.6,14.3,17.9,71.4,10.7
4,Angola,Angola,24,540.0,AGO,24.0,18.75,15.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<b> Question 2.1 </b> <br>
Recode 2 different variables into new categories. They can both be continuous-ish or both be nominal-ish, or
one of each. Tell me what you did and explain the variable(s)

The 2 variables I want to recode is Serverity of Conflict and Human Development Index. From this data I expect to see some correlation with how servere the conflict is within a country, that could be one of the factors that explains the Human Development Infex of a Country. Either high conflict and Low HDI or low conflict and HDI production. 

In [56]:
#First Variable
#0-1 = No Conflict, 2-4 = Minor Conflict, 5-7 = Major Conflict, Servere Conflict
conditions = [
    (df['bti_ci'] <= 1),
    (df['bti_ci'] >= 2) & (df['bti_ci'] <= 4),
    (df['bti_ci'] >= 5) & (df['bti_ci'] <= 7),
    (df['bti_ci'] >= 8) & (df['bti_ci'] <= 10)
]
choices = ['No Conflict', 'Minor Conflict', 'Major Conflict', 'Servere Conflict']
df['conflict_level'] = np.select(conditions, choices, default=np.nan)

# Ensure missings are treated as NaN
df['conflict_level'] = df['conflict_level'].replace('nan', np.nan)

#Order
df['conflict_level'] = pd.Categorical(df['conflict_level'], categories=choices, ordered=True)

#Preview output: 
df.conflict_level.value_counts(dropna=False)

conflict_level
NaN                 59
Minor Conflict      57
Major Conflict      54
Servere Conflict    16
No Conflict          8
Name: count, dtype: int64

We categorized each country based on a scale provide by QOG that dedicated what the scale of 1-10 represented for each country. This data comes from Bertelsmann Transformation Index, and measures how serious social ethnic and relgious conflicts are. 

Important to Note that we are missing data from 59 countries. That is more than I had antipicated. 

In [57]:
#Second Variable
df['undp_hdi'].describe()

count    190.000000
mean       0.719100
std        0.153282
min        0.380000
25%        0.599250
50%        0.737500
75%        0.839000
max        0.965000
Name: undp_hdi, dtype: float64

In [58]:
#Second Variable
# # < 0.50 = Low ,0.50–0.7 = Medium, 0.7– 0.84 = High, ≥ 0.85 = Very High
conditions = [
    (df['undp_hdi'] < 0.6),
    (df['undp_hdi'] >= 0.6) & (df['undp_hdi'] < 0.74),
    (df['undp_hdi'] >= 0.74) & (df['undp_hdi'] < 0.84),
    (df['undp_hdi'] >= 0.84)
]

choices = [
    'Low HDI',
    'Medium HDI',
    'High HDI',
    'Very High HDI'
]
df['hdi_level'] = np.select(conditions, choices, default=np.nan)

# Ensure missings are treated as NaN
df['hdi_level'] = df['hdi_level'].replace('nan', np.nan)

df['hdi_level'] = pd.Categorical(df['hdi_level'], categories=choices, ordered=True)


#Preview output: 
df.hdi_level.value_counts(dropna=False)


hdi_level
Medium HDI       49
Low HDI          48
Very High HDI    48
High HDI         45
NaN               4
Name: count, dtype: int64

Utilizing df['undp_hdi'].describe(), we were able to split the data of HDI into 4 quartiles of categories based on the data. This does roughly correlate with the official values of the scale provided by the UN. I 

<b> Question 2.2 </b> <br>
2-- Use one (or both) of your recoded variables to do a cross-tabulation. Explain your results.

In [59]:
pd.crosstab(df['conflict_level'], df['hdi_level'], dropna=False)

hdi_level,Low HDI,Medium HDI,High HDI,Very High HDI
conflict_level,,,,
No Conflict,0,2,3,3
Minor Conflict,11,15,16,13
Major Conflict,23,15,13,3
Servere Conflict,12,1,3,0


When Cross tabulating the two variables, you can observe that the distrubution of conflict serverity decreases the higher the HDI is. This does makes sense as countries are more likely to be able to work towards a higher quality of life if there is no conflict or less conflict within their country. They are able to focus on improving quality of life rather than focusing on the matters that arise from having conflict within the country. 

<b> Question 2.3 </b> <br>
3-- Run a correlation of one variable with another variable; make all of the recodes necessary to make the
correlation as easy to interpret as possible; and explain your results.

We are will be using the World Uncertainty Index. To make the WUI comparable across countries, authors scale the raw counts by the total number of words in each report — the number of “uncertainty” words per thousand words.

In [84]:
# Recode variable 1 to match variable 2 scale
df['bti_ci_scaled'] = df['bti_ci'] / 10

# Drop missing values from both variables
df_corr = df.dropna(subset=['bti_ci_scaled', 'undp_hdi'])

# Pearson correlation undp_hdi
correlation = df_corr['undp_hdi'].corr(df_corr['bti_ci_scaled'])
print(f"Correlation between scaled BTI Civil Index and HDI: {correlation:.3f}")

Correlation between scaled BTI Civil Index and HDI: -0.523


My correlation is -0.0.523. This tells me that there is a strong correlation between countries with higher conflict end to have a lower HDI. This would make sense based on the previous assumptions.  

<b> Question 2.4 </b> <br>
4-- Identify the most extreme cases on some variable. Interpret the results.

In [ ]:
Based on Question 2.2, I was curious who were the three countries that had major conflict and a very high HDI

In [93]:
final_df = df[(df['conflict_level'] == 'Major Conflict') & (df['hdi_level'] == 'Very High HDI')]
final_df[['cname', 'conflict_level', 'hdi_level']].head(10)

,cname,conflict_level,hdi_level
11,Bahrain,Major Conflict,Very High HDI
35,Chile,Major Conflict,Very High HDI
176,Turkey,Major Conflict,Very High HDI


Those three countries are Bahrain, Chile and Turkey. Sounds like i need to brush up on my news. 